<a href="https://colab.research.google.com/github/youssouf2230/Base-de-RAG/blob/main/rag_chap2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install --upgrade pip
!pip install llama-index llama-index-llms-mistralai llama-index-embeddings-huggingface langchain sentence-transformers

In [ ]:
from google.colab import userdata
mistral_key_api=userdata.get('mistral_key_api')
print(mistral_key_api)

gKrvir2uCmqcVDVLqWXU875bXR76Usnw


In [ ]:
from llama_index.core import VectorStoreIndex, Document
from llama_index.llms.mistralai import MistralAI
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

#  Embeddings locaux
embed_model = HuggingFaceEmbedding(model_name="all-MiniLM-L6-v2")

#  Mistral LLM
llm = MistralAI(api_key=mistral_key_api)

#  Tes documents
documents = [
    Document(text="Pepe a pris sa retraite en 2025."),
    Document(text="Ronaldo a annoncé son retrait officiel en 2024."),
    Document(text="Mbappé envisage de prendre sa retraite vers 2032."),
    Document(text="Messi a pris sa retraite internationale en 2023."),
    Document(text="Neymar pourrait se retirer à la fin de la saison 2026."),
]

#  Construction du RAG
index = VectorStoreIndex(documents, embed_model=embed_model)
query_engine = index.as_query_engine(llm=llm,simulate=True)

#  Requêtes
print(query_engine.query("Quand peut-on prendre sa retraite ?"))
print(query_engine.query("Qui peut prendre sa retraite ?"))
print(query_engine.query("Quelle est la date limite pour prendre sa retraite ?"))


La retraite peut être prise à différents moments selon les individus. Par exemple, certains choisissent de mettre fin à leur carrière vers le début des années 2030, tandis que d'autres peuvent le faire plus tôt, comme autour de la moitié des années 2020. Le moment dépend donc des préférences et des décisions personnelles.
Kylian Mbappé peut prendre sa retraite.
La date limite envisagée pour prendre sa retraite est **aux alentours de 2032**.


In [ ]:
# lire un document depuis google
import requests

# 1. Télécharger le fichier PDF en local
pdf_url = "https://static.tlscontact.com/media/ma/cas/be/2019_08_liste_harmonisee_documents_a_soumettre_pour_un_visa_de_court_sejour.pdf"
pdf_path = "visa_court_sejour.pdf"

with open(pdf_path, "wb") as f:
    f.write(requests.get(pdf_url).content)

# 2. Lire avec PDFReader
from llama_index.readers.file import PDFReader

loader = PDFReader()
documents = loader.load_data(file=pdf_path)

# 3. Tu peux vérifier
print(f"{len(documents)} document(s) chargés.")
print(documents[0].text[:500])


8 document(s) chargés.
Liste des documents justificatifs devant être produits par les demandeurs de visa de 
court séjour au Maroc  
 
I. Documents à présenter par tous les demandeurs de visa  
  
1. Preuve du moyen de transport  
- réservation d'un billet aller-retour (avion, autobus ou bateau), selon le cas.  
2. Preuve d'hébergement  
- réservation(s) d'hôtel, ou  
- preuve que le demandeur de visa loue ou possède des biens immobiliers dans l’État 
membre de destination, ou  
- confirmation de l’hébergement chez un


In [ ]:
from llama_index.memory import ConversationBufferMemory

memory = ConversationBufferMemory(memory_key="chat_history")

# Embeddings locaux
embed_model = HuggingFaceEmbedding(model_name="all-MiniLM-L6-v2")

# Mistral LLM
llm = MistralAI(api_key=mistral_key_api)

# Construire l’index à partir des documents PDF
index = VectorStoreIndex(documents, embed_model=embed_model)

# Créer le moteur de requête (simulate=True si tu veux simuler)
query_engine = index.as_query_engine(llm=llm, simulate=True,memory=memory)

# Test de requêtes
print(query_engine.query("Quels documents sont nécessaires pour un visa de court séjour ?"))
print(query_engine.query("Quelles sont les conditions pour déposer une demande ?"))
